In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np
import pandas as pd

In [4]:
#Loading dataset
train1 = pd.read_parquet('dataset/train-00000-of-00002.parquet', engine='pyarrow')
train2 = pd.read_parquet('dataset/train-00001-of-00002.parquet', engine='pyarrow')
test = pd.read_parquet('dataset/test-00000-of-00001.parquet', engine='pyarrow')
validation = pd.read_parquet('dataset/validation-00000-of-00001.parquet', engine='pyarrow')

In [5]:
# Concatenate train1 and train 2
train = pd.concat([train1, train2], axis=0, ignore_index=True)